In [ ]:
import keras
import numpy as np
import pandas as pd 
import os
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow.keras
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import math

In [ ]:
x = []
label = []
x_validation = []
label_validation = []
x_test = []
label_test = []

covid_test = []
covid_valid = []
covid_train = []
pneumonia_test = []
pneumonia_valid = []
pneumonia_train = []
normal_test = []
normal_valid = []
normal_train = []

images = []
img_size = 299

#valid_covid = "../input/radiography-dataset/new_Dataset/test/Covid"
covid = "../input/covid19-radiography-database/COVID-19_Radiography_Dataset/COVID"
covid_list = os.listdir(covid)

pneumonia = "../input/covid19-radiography-database/COVID-19_Radiography_Dataset/Viral Pneumonia"
pneumonia_list = os.listdir(pneumonia)

normal = "../input/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal"
normal_list = os.listdir(normal)


csv_file_link = "../input/radiography-database-springer-2-image-division/division_into_train_valid__test.csv"
csv_file = pd.read_csv(csv_file_link)
# print(csv_file.columns)


group_by = csv_file.groupby('image_type')
for i,j in group_by:
    print(i)
    
    usedin_group_by = j.groupby('used_in')
    for k,l in usedin_group_by:
        
        print(k)
        print(len(l))
        for m in range(0, len(l)):
            if i == 'covid':
                    
                    link = covid + '/' + l.iloc[m]['image_name']
                    img = cv2.imread(link)
                    image = cv2.resize(img, ( img_size , img_size ))
            
                    if k == 'test':
                        x_test.append(image)
                        label_test.append([0])
                        covid_test.append(image)
                    elif k == 'validation':
                        x_validation.append(image)
                        label_validation.append([0])
                        covid_valid.append(image)
                    else:
                        x.append(image)
                        label.append([0])
                        covid_train.append(image)

                       
            if i == 'pneumonia':
                    
                    link = pneumonia + '/' + l.iloc[m]['image_name']
                    img = cv2.imread(link)
                    image = cv2.resize(img, ( img_size , img_size ))
                    if k == 'test':
                        x_test.append(image)
                        label_test.append([1])
                        pneumonia_test.append(image)
                    elif k == 'validation':
                        x_validation.append(image)
                        label_validation.append([1])
                        pneumonia_valid.append(image)
                    else:
                        x.append(image)
                        label.append([1])
                        pneumonia_train.append(image)
                    
                        
            if i == 'normal':
                    
                    link = normal + '/' + l.iloc[m]['image_name']
                    img = cv2.imread(link)
                    image = cv2.resize(img, ( img_size , img_size ))
                    
                    if k == 'test':
                        x_test.append(image)
                        label_test.append([2])
                        normal_test.append(image)
                    elif k == 'validation':
                        x_validation.append(image)
                        label_validation.append([2])
                        normal_valid.append(image)
                    else:
                        x.append(image)
                        label.append([2])
                        normal_train.append(image)
                    
 
                        
x = np.asarray(x)
x_validation = np.asarray(x_validation)
x_test = np.asarray(x_test)
label = keras.utils.np_utils.to_categorical(label , 3 )
#label = np.array(label)
label_validation = keras.utils.np_utils.to_categorical(label_validation , 3 )
#print(label_validation)
#label_validation = np.array(label_validation)
label_test = keras.utils.np_utils.to_categorical(label_test , 3 )
#print(label_test)


print("total_train", len(x))
print("total_train",len(label))
print("total_valid",len(x_validation))
print("total_valid",len(label_validation))
print("total_test",len(x_test))
print("total_test",len(label_test ))
print()
print("covid_train = ", len(covid_train)) 
print("pneumonia_train = ", len(pneumonia_train)) 
print("normal_train = ", len(normal_train)) 
print("covid_valid = ", len(covid_valid)) 
print("pneumonia_valid = ", len(pneumonia_valid)) 
print("normal_valid = ", len(normal_valid)) 
print("covid_test = ", len(covid_test)) 
print("pneumonia_test = ", len(pneumonia_test)) 
print("normal_test = ", len(normal_test)) 


In [ ]:
model_path = '../input/final-comparison-0827/Updated_3class_Xception_50_epoch_Nikolaou.h5'
# model_path = '../input/archive2-train-valid-test-h5-csv/ResNet50_springer_paper_2.h5'

extract_saved_model = tf.keras.models.load_model(model_path)
extract_saved_model.summary()

In [ ]:
Validation_score = extract_saved_model.evaluate(x_validation, label_validation, verbose=2)
print('Restored model, Validation_accuracy: {:5.2f}%'.format(100 * Validation_score[1]))

In [ ]:
score = extract_saved_model.evaluate(x_test, label_test, verbose=2)
print('Restored model, testing_accuracy: {:5.2f}%'.format(100 * score[1]))
# print(extract_saved_model.predict(x_test).shape)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
n_classes = 3
predict_proba = extract_saved_model.predict(x_test)
print(predict_proba)
print(len(predict_proba))
# print(predict_proba[0])

In [ ]:
# predict_class = predict_proba[2].argmax(axis=-1)      ######### for InceptionV1_archive2
# predict_class = predict_proba[0].argmax(axis=-1)        ######### for InceptionV3_archive2
predict_class = predict_proba.argmax(axis=-1)
# print(predict_class)
true_class = label_test.argmax(axis=-1)
# print(true_class)
predict_classes = list(predict_class)
true_classes = list(true_class)
print(classification_report(true_classes, predict_classes, target_names=['Covid', 'Pneumonia', 'Normal']))

confused_matrix = confusion_matrix(true_classes, predict_classes, labels=range(n_classes))
print(confused_matrix)

In [ ]:
# precision =  [0.95217391 0.90877193 0.9649635 ]  Recall =  [0.90871369 0.96282528 0.97301276]  F1-score =  [0.92993631 0.93501805 0.96897141]
# Covid: accuracy =  [0.96732673 0.98811881 0.95808581]  sensitivity =  [0.90871369 0.96282528 0.97301276]  specificity =  [0.98569571 0.99058312 0.92741935]
cm = confused_matrix
TP = np.diag(cm)
FP = np.sum(cm, axis=0) - TP
FN = np.sum(cm, axis=1) - TP
num_classes = 3
TN = []
for i in range(num_classes):
    temp = np.delete(cm, i, 0)    # delete ith row
    temp = np.delete(temp, i, 1)  # delete ith column
    TN.append(sum(sum(temp)))
    
print("Covid: TP = ", TP, " TN = ",TN, " FP = ",FP, " FN = ",FN)
accuracy = (TP+TN)/(TP+TN+FP+FN)
accuracy = [i * 100 for i in accuracy]
precision = TP/(TP+FP)
precision = [i * 100 for i in precision]
recall = TP/(TP+FN)
recall = [i * 100 for i in recall]
sensitivity = TP / (TP + FN)
sensitivity = [i * 100 for i in sensitivity]
specificity = TN/(TN+FP)
specificity = [i * 100 for i in specificity]
Fscore = 2*TP /(2*TP + FP + FN)
Fscore = [i * 100 for i in Fscore]
print("   precision = ",np.around(precision, 2), " Recall = ",np.around(recall, 2), " F1-score = ",np.around(Fscore, 2))
print("      accuracy = ",np.around(accuracy, 2), " sensitivity = ",np.around(sensitivity, 2), " specificity = ",np.around(specificity, 2))

In [ ]:
print("Covid & ",np.around(accuracy[0], 2)," & ",np.around(sensitivity[0], 2)," & ",np.around(specificity[0], 2)," & ",np.around(precision[0], 2)," & ",np.around(recall[0], 2)," & ",np.around(Fscore[0], 2))
print("Pneumonia & ",np.around(accuracy[1], 2)," & ",np.around(sensitivity[1], 2)," & ",np.around(specificity[1], 2)," & ",np.around(precision[1], 2)," & ",np.around(recall[1], 2)," & ",np.around(Fscore[1], 2))
print("Normal & ",np.around(accuracy[2], 2)," & ",np.around(sensitivity[2], 2)," & ",np.around(specificity[2], 2)," & ",np.around(precision[2], 2)," & ",np.around(recall[2], 2)," & ",np.around(Fscore[2], 2))

print("& Covid-19 &  ",np.around(precision[0], 2)," & ",np.around(recall[0], 2)," & ",np.around(Fscore[0], 2),
      " & & & Pneumonia & ",np.around(precision[1], 2)," & ",np.around(recall[1], 2)," & ",np.around(Fscore[1], 2), 
        " & & & Normal & ",np.around(precision[2], 2)," & ",np.around(recall[2], 2)," & ",np.around(Fscore[2], 2))

# **Micro Averages**
Micro Average Precision is the sum of all true positives and divides by the sum of all true positives plus the sum of all false positives. So basically you divide the number of correctly identified predictions by the total number of predictions.
https://androidkt.com/micro-macro-averages-for-imbalance-multiclass-classification/

In [ ]:
cnf_matrix = confused_matrix
FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix) 
FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
TP = np.diag(cnf_matrix)
TN = cnf_matrix.sum() - (FP + FN + TP)
FP = FP.astype(float)
FN = FN.astype(float)
TP = TP.astype(float)
TN = TN.astype(float)
TFP = FP[0] + FP[1] + FP[2]
TFN = FN[0] + FN[1] + FN[2]
TTP = TP[0] + TP[1] + TP[2]
TTN = TN[0] + TN[1] + TN[2]
# Overall accuracy
ACC = ((TTP+TTN)/(TTP+TFP+TFN+TTN))*100
sensitivity = (TTP / (TTP + TFN))*100
specificity = (TTN/(TTN+TFP))*100
PPV = (TTP/(TTP + TFP))*100
NPV = (TTN/(TTN + TFN))*100
Fscore = (2*TTP /(2*TTP + TFP + TFN))*100

print('ACC: %.2f' % ACC)
print('sensitivity: %.2f' % sensitivity)
print('specificity: %.2f' % specificity)
print('PPV: %.2f' % PPV)
print('NPV: %.2f' % NPV)
print('NPV: %.2f' % Fscore)
print(img_size," & ",'%.2f' % ACC," & ",'%.2f' % sensitivity," & ",'%.2f' % specificity,
      " & ",'%.2f' % PPV, " & ",'%.2f' % NPV, " & ",'%.2f' % Fscore)

In [ ]:
# INCEPTION_V2_128_tune0
classes = ["0", "1", "2"]
df_confusion_matrix = pd.DataFrame(confused_matrix, index = classes, columns = classes)

axis_label = ['Covid', 'Pneumonia', 'Normal']
fig, ax = plt.subplots(figsize=(8,6))
sns.set(font_scale=1.4)
cfm_plot = sns.heatmap(confused_matrix, annot=True, linewidths=1, ax=ax, fmt='d', cmap='Blues')
ax.set_xlabel('Predicted labels', fontdict=dict(weight='bold'), fontsize=20)
ax.set_ylabel('True labels', fontdict=dict(weight='bold'), fontsize=20) 
# ax.set_title('Confusion Matrix For Inception_V2'); 
ax.set_xticklabels(axis_label, fontsize=18)
ax.set_yticklabels(axis_label, rotation=0, ha='right', fontsize=18)


cfm_plot.figure.savefig("./Xception_ConfusionMatrix_new.png", bbox_inches='tight')

In [ ]:
epochs = 60    
# epochs = 100
epochs_range = range(epochs)

csv_file = pd.read_csv('../input/final-comparison-0827/Xception_radiography_springer_paper_2_50_epoch.csv')
# csv_file = pd.read_csv('../input/archive2-train-valid-test-h5-csv/ResNet50_springer_paper_2.csv')

# acc = csv_file['acc']
acc = csv_file['acc']
acc = list(acc)
# val_acc = csv_file['val_acc']
val_acc = csv_file['val_acc']
val_acc = list(val_acc)
# loss = csv_file['loss']
loss = csv_file['loss']
loss = list(loss)
# val_loss = csv_file['val_loss']
val_loss = csv_file['val_loss']
val_loss = list(val_loss)
print(loss)

In [ ]:
# epochs = 100
# epochs_range = range(epochs)

plt.figure(figsize=(8, 5))
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
# plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch', weight='bold')
plt.ylabel('Accuracy Value', weight='bold')
plt.savefig("Xception_Accuracy_labeling_new.png")
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(epochs_range, loss , label='Training Loss')
plt.plot(epochs_range, val_loss , label='Validation Loss')
plt.legend(loc='upper right')
# plt.title('Training and Validation Loss')
plt.xlabel('Epoch', weight='bold')
plt.ylabel('Loss Value', weight='bold')
plt.savefig("Xception_loss_labeling_new.png")
plt.show()